# **Installing the required libraries**

In [ ]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers>=4.41.0 einops accelerate langchain bitsandbytes sentence-transformers
!pip install sentence-transformers
%pip install llama-index-llms-huggingface
%pip install llama-index-llms-huggingface-api
!pip install langchain_community
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor
%pip install llama-index-embeddings-langchain
!pip install llama-index-readers-file


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.3.1
    Uninstalling sentence-transformers-3.3.1:
      Successfully uninstalled sentence-transformers-3.3.1


In [ ]:
!pip install llama-index --upgrade


In [ ]:
!pip install Flask


In [ ]:
!pip install flask-ngrok pyngrok


In [ ]:
!pip install flask-ngrok
!pip install flask-cors
!pip install flask
!pip install pyngrok


In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.prompts.prompts import SimpleInputPrompt

# **Data Retrieval**

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

#API key
api_key = "387f971f-0b7e-4bf2-9a5c-2bba3a2c638f"

#Particular Sections to fetch
sections = ["technology", "science", "sports", "health", "education"]

#Fetching Articles of last 1 month
from_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
to_date = datetime.now().strftime("%Y-%m-%d")

#Endpoint URL
base_url = "https://content.guardianapis.com/search"

# Function to Fetch Articles
def fetch_articles(section, page=1):
    params = {
        "api-key": api_key,
        "section": section,
        "from-date": from_date,
        "to-date": to_date,
        "show-fields": "body",
        "page-size": 50,
        "page": page
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json().get("response", {})
    else:
        print(f"Failed to fetch articles for section {section}. Status code: {response.status_code}")
        return {}

In [ ]:
# Fetching and Agregating the Articles
all_articles = []

for section in sections:
    print(f"Fetching articles from section: {section}")
    page = 1
    while True:
        data = fetch_articles(section, page)
        results = data.get("results", [])
        if not results:
            break
        all_articles.extend(results)
        if not data.get("pages", 1) > page:
            break
        page = page+1

Fetching articles from section: technology
Fetching articles from section: science
Fetching articles from section: sports
Fetching articles from section: health
Fetching articles from section: education


In [ ]:
#Processing the Articles
if all_articles:
    articles_data = []
    for article in all_articles:
        title = article.get("webTitle", "N/A")
        section = article.get("sectionName", "N/A")
        url = article.get("webUrl", "N/A")
        published_date = article.get("webPublicationDate", "N/A")
        content = article.get("fields", {}).get("body", "Content not available")

        articles_data.append({
            "Title": title,
            "Section": section,
            "URL": url,
            "Published Date": published_date,
            "Content": content
        })


In [ ]:
#Converting the data to DatafRame
import pandas as pd
df = pd.DataFrame(articles_data)
print(f"Fetched {len(df)} articles from the past month.")
print(df.head())  # Display the first few articles


Fetched 221 articles from the past month.
                                               Title     Section  \
0  Who banned TikTok? Politicians toss culpabilit...  Technology   
1  Should I be worried about my obsessive TikTok ...  Technology   
2              Would you let AI choose your outfits?  Technology   
3  ‘Young women can fall pregnant very easily’: i...  Technology   
4                   How does the US Tiktok ban work?  Technology   

                                                 URL        Published Date  \
0  https://www.theguardian.com/technology/2025/ja...  2025-01-19T14:00:06Z   
1  https://www.theguardian.com/technology/2025/ja...  2025-01-19T13:00:02Z   
2  https://www.theguardian.com/technology/2025/ja...  2025-01-19T13:00:02Z   
3  https://www.theguardian.com/technology/2025/ja...  2025-01-19T10:00:03Z   
4  https://www.theguardian.com/technology/2025/ja...  2025-01-19T05:19:12Z   

                                             Content  
0  <p>The United States o

In [ ]:
df.shape

(221, 5)

# **Text Processing**

**Parsing HTML content**

In [ ]:
#Creating a function to parse HTML content and extrating text
def parse_html_content(html_content):
    return BeautifulSoup(html_content, "html.parser").get_text(separator=' ', strip=True)

#Applying the fn to Content column
df["Content"] = df["Content"].apply(parse_html_content)

print(df['Content'].head(2))

0    The United States of America deleted TikTok ea...
1    As the US TikTok ban looms , users of the app ...
Name: Content, dtype: object


**Removing StopWords from content**

In [ ]:
#Creating a function to remove StopWords from the Content column
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

#Fn to remove stopwords from Content column
def remove_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

#Applying the fn to remove stopwords from Content column
df["Content"] = df["Content"].apply(remove_stopwords)
df['Content'].head(2)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,Content
0,United States America deleted TikTok early mor...
1,"US TikTok ban looms , users app posting farewe..."


In [ ]:
#Creating a fn to convert Content text to lower case
def clean_text(text):
  text = text.lower()
  return text

df['Content'] = df['Content'].apply(clean_text)
df['Content'].head(2)

,Content
0,united states america deleted tiktok early mor...
1,"us tiktok ban looms , users app posting farewe..."


**Extracting Metadata and storing all the content and metadata in a variable named document**

In [ ]:
from llama_index.core import Document
document = []
for index, row in df.iterrows():
  parsed_text = BeautifulSoup(row['Content'], 'html.parser').get_text(separator=' ', strip=True)
  document.append(Document(
  text=parsed_text,
  metadata={
      "Title": row['Title'],
      "Section": row['Section'],
      "Url": row['URL'],
      "date": row['Published Date']
      }
  ))

In [ ]:
document[0:2]

[Document(id_='2bf5934e-de06-485d-9381-1819dcb17201', embedding=None, metadata={'Title': 'Who banned TikTok? Politicians toss culpability like a football', 'Section': 'Technology', 'Url': 'https://www.theguardian.com/technology/2025/jan/19/tiktok-ban-culpability', 'date': '2025-01-19T14:00:06Z'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='united states america deleted tiktok early morning 19 january . government formed “ people , people ” , words abraham lincoln , made scant evidence available people . power 11th hour realize unpopular paternalistic move might , best lay blame others . us ban app used beloved 170 million americans ? fear china ’ propaganda data collection . ’ far-reaching , unprecedented move . text protecting americans foreign adversary controlled applications act , passed april signed joe biden , reads : “ b

# **Creating Search Workflow using LLAMA INDEX + RAG + LANGCHAIN for Content-based filtering**

**Setting up System prompt and the format supported by LLM (LLama2)**

In [ ]:
system_prompt = """
You are a recommendation search assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided.
For each result, provide the article title, URL, publication date, and a brief summary. Ensure the metadata is clearly displayed for reference.
"""

#default format supported by llama2
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

**Hugging Face api login**

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `sssss` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

**LLM MODEL (LLama2)**

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=500,
    generate_kwargs={"temperature": 0.5, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float16}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

**Embeddings**

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

lc_embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
embed_model = LangchainEmbedding(lc_embed_model)

<ipython-input-19-e7d904cd992f>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  lc_embed_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Settings**

In [ ]:
from llama_index.core import ServiceContext
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
node_parser = SentenceSplitter(chunk_size=900, chunk_overlap=50)

In [ ]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = node_parser


In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings

Settings.transformations = [SentenceSplitter(chunk_size=900)]

**Indexing**

In [ ]:
index = VectorStoreIndex.from_documents(
    document, embed_model=embed_model
)

query_engine = index.as_query_engine(llm=llm)

In [ ]:
# Save the embeddings object using pickle
import pickle
with open('vector_store_index.pkl', 'wb') as file:
    pickle.dump(index, file)


In [ ]:
#Saving the embeddings local
from llama_index.core.storage import StorageContext

# Assuming `index` is your VectorStoreIndex
storage_context = index.storage_context

# Save the index locally
storage_context.persist(persist_dir="./index")

In [ ]:
import pickle

# Load the index object from the file
with open('/content/vector_store_index.pkl', 'rb') as file:
    loaded_index = pickle.load(file)


**Asing search recommendations from the bot**

In [ ]:
response=query_engine.query("Retrieve top articles related to 'Sentiment analysis in finance'.")
print(response)


KeyboardInterrupt: 

# **API Integration**

In [ ]:
%%writefile index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Article Search Engine</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f7fc;
            margin: 0;
            padding: 0;
        }
        .container {
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
            background-color: white;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        h1 {
            text-align: center;
            color: #333;
        }
        .search-box {
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 20px;
        }
        .search-box input {
            width: 80%;
            padding: 10px;
            font-size: 16px;
            border-radius: 5px;
            border: 1px solid #ddd;
        }
        .search-box button {
            padding: 10px 20px;
            background-color: #007BFF;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        .search-box button:hover {
            background-color: #0056b3;
        }
        .result {
            padding: 15px;
            background-color: #f9f9f9;
            margin-bottom: 15px;
            border-radius: 5px;
            border: 1px solid #ddd;
        }
        .result h3 {
            margin: 0;
            font-size: 20px;
        }
        .result p {
            color: #555;
        }
    </style>
</head>
<body>

    <div class="container">
        <h1>Article Search Engine</h1>
        <div class="search-box">
            <input type="text" id="queryInput" placeholder="Enter your query here..." />
            <button id="searchButton" onclick="searchArticles()">Search</button>
        </div>

        <div id="resultsContainer"></div>
    </div>

    <script>
        function searchArticles() {
            const query = document.getElementById("queryInput").value;
            if (query.trim() === "") {
                alert("Please enter a search query.");
                return;
            }

            // Make a POST request to the Flask API
            fetch("http://127.0.0.1:5000/query", {
                method: "POST",
                headers: {
                    "Content-Type": "application/json"
                },
                body: JSON.stringify({ query: query })
            })
            .then(response => response.json())
            .then(data => {
                // Display the results
                const resultsContainer = document.getElementById("resultsContainer");
                resultsContainer.innerHTML = ""; // Clear any previous results

                if (data.error) {
                    resultsContainer.innerHTML = `<p>Error: ${data.error}</p>`;
                } else if (data.response && data.response.length > 0) {
                    data.response.forEach(article => {
                        const articleDiv = document.createElement("div");
                        articleDiv.classList.add("result");
                        articleDiv.innerHTML = `
                            <h3>${article.title}</h3>
                            <p>${article.summary}</p>
                            <a href="${article.url}" target="_blank">Read more...</a>
                        `;
                        resultsContainer.appendChild(articleDiv);
                    });
                } else {
                    resultsContainer.innerHTML = "<p>No articles found for this query.</p>";
                }
            })
            .catch(error => {
                console.error("Error fetching articles:", error);
                alert("There was an error processing your request.");
            });
        }
    </script>

</body>
</html>


Writing index.html


In [ ]:
import os

# Create a templates folder
os.makedirs('/content/templates', exist_ok=True)


In [ ]:
from google.colab import files

# Upload the index.html file
uploaded = files.upload()

# Move the uploaded index.html to the templates folder
os.rename('index.html', '/content/templates/index.html')

Saving index (1).html to index (1).html


In [ ]:
from flask import Flask, request, render_template, jsonify
from flask_cors import CORS
import pickle
from llama_index.core import VectorStoreIndex
from pyngrok import ngrok

app = Flask(__name__, template_folder='/content/templates')  # Specify the template folder path
CORS(app)

# Load the index and recreate the query engine
with open('vector_store_index.pkl', 'rb') as f:
    index = pickle.load(f)

# Assume `llm` is already defined or initialize it here
query_engine = index.as_query_engine(llm=llm)

@app.route('/')
def home():
    # Serve the index.html from the specified template folder
    return render_template('index.html')

@app.route('/add_article', methods=['POST'])
def add_article():
    try:
        article_data = request.json
        # Handle adding the article to your database or storage
        return jsonify({"status": "success", "data": article_data})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/perform_inference', methods=['POST'])
def perform_inference():
    try:
        query_data = request.json
        # Perform inference based on query_data
        response = {"result": "inference result"}
        return jsonify(response)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    # Setup a public URL with ngrok
    public_url = ngrok.connect(5000)
    print(f" * Ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")
    # Run the Flask app
    app.run(port=5000)


 * Ngrok tunnel "NgrokTunnel: "https://5c58-35-230-113-221.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 11:19:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 11:19:03] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
import pickle
from llama_index.core import VectorStoreIndex
from pyngrok import ngrok
from transformers import pipeline

# Initialize Flask app
app = Flask(__name__, template_folder='/content/templates')  # Specify the template folder path
CORS(app)

# Load the index from pickle file and recreate the query engine
with open('vector_store_index.pkl', 'rb') as f:
    index = pickle.load(f)

# Load the Llama 2 model from Hugging Face (replace with the correct Llama 2 model name)
# Example with a Llama 2 model (you may replace with your specific model)
llm = pipeline("text-generation", model="meta-llama/Llama-2-7b-hf")

# Recreate the query engine
query_engine = index.as_query_engine(llm=llm)

@app.route('/')
def home():
    return render_template('index.html')  # Serve the index.html from the specified template folder

@app.route('/perform_inference', methods=['POST'])
def perform_inference():
    try:
        query_data = request.json.get('query')  # Get the query from the request
        if not query_data:
            return jsonify({"error": "Query is required"}), 400

        # Perform inference using query engine
        response = query_engine.query(query_data)

        return jsonify({"result": response.response})  # Return the response from the query engine
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    # Setup a public URL with ngrok
    public_url = ngrok.connect(5000)
    print(f" * Ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

    # Run the Flask app
    app.run(port=5000)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-2ed463c51ee2>", line 18, in <cell line: 0>
    llm = pipeline("text-generation", model="meta-llama/Llama-2-7b-hf")
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/__init__.py", line 940, in pipeline
    framework, model = infer_framework_load_model(
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py", line 289, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py", line 564, in from_pretrained
    return model_class.from_p

TypeError: object of type 'NoneType' has no len()

In [ ]:
import os

# Create static folder if not already present
os.makedirs('static', exist_ok=True)

# Move index.html to the static folder
os.rename('index.html', 'static/index.html')

In [ ]:
from flask import Flask, request, render_template, jsonify
from flask_cors import CORS
import pickle
from llama_index.core import VectorStoreIndex
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)

# Load the index and recreate the query engine
with open('vector_store_index.pkl', 'rb') as f:
    index = pickle.load(f)

# Assume `llm` is already defined or initialize it here
query_engine = index.as_query_engine(llm=llm)

@app.route('/')
def home():
    # Serve the index.html from the static folder
    return render_template('index.html')

@app.route('/add_article', methods=['POST'])
def add_article():
    try:
        article_data = request.json
        # Handle adding the article to your database or storage
        return jsonify({"status": "success", "data": article_data})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/perform_inference', methods=['POST'])
def perform_inference():
    try:
        query_data = request.json
        # Perform inference based on query_data
        response = {"result": "inference result"}
        return jsonify(response)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    # Setup a public URL with ngrok
    public_url = ngrok.connect(5000)
    print(f" * Ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")
    # Run the Flask app
    app.run(port=5000)

 * Ngrok tunnel "NgrokTunnel: "https://220b-35-230-113-221.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask_cors/extension.py", line 194, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
    

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2rqQ93keiuc7AxVOokV5NxPgssH_3Kx2iefrS6S3gsiLD6a4C")

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pickle
from llama_index.core import VectorStoreIndex
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)

# Load the index and recreate the query engine
with open('vector_store_index.pkl', 'rb') as f:
    index = pickle.load(f)

# Assume `llm` is already defined or initialize it here
query_engine = index.as_query_engine(llm=llm)

@app.route('/')
def home():
    return "Welcome to the Query API! Use '/query' endpoint to ask questions."

@app.route('/query', methods=['POST'])
def query():
    try:
        user_query = request.json.get('query')
        if not user_query:
            return jsonify({"error": "No query provided"}), 400
        response = query_engine.query(user_query)
        return jsonify({"query": user_query, "response": response.response})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    # Setup a public URL with ngrok
    public_url = ngrok.connect(5000)
    print(f" * Ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")
    # Run the Flask app
    app.run(port=5000)

 * Ngrok tunnel "NgrokTunnel: "https://b8f6-35-230-113-221.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 10:58:09] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 10:58:10] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
from flask import Flask, request, jsonify
from datetime import datetime
import torch

# Initialize Flask app
app = Flask(__name__)

# Real-time indexing: Add articles to the database
@app.route("/articles", methods=["POST"])
def add_article():
    try:
        # Get JSON data from the request
        data = request.get_json()

        # Extract relevant fields
        title = data.get("title")
        section = data.get("section")
        url = data.get("url")
        published_date = data.get("published_date")
        content = data.get("content")

        # Create a Document object for the article
        document.append(Document(
            text=content,
            metadata={
                "Title": title,
                "Section": section,
                "Url": url,
                "date": published_date
            }
        ))

        # Optionally, reindex the documents after adding a new one
        index.add_documents([document[-1]])

        return jsonify({"message": "Article added and indexed successfully!"}), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 400

In [ ]:
# Searching articles: Query articles with optional filters
@app.route("/search", methods=["GET"])
def search_articles():
    try:
        query = request.args.get("query", "")  # The search query
        date_filter = request.args.get("date", None)  # Optional date filter
        section_filter = request.args.get("section", None)  # Optional section filter
        sentiment_filter = request.args.get("sentiment", None)  # Optional sentiment filter

        # Start with the user's query as the search query
        search_query = query

        # Apply filters to the query if provided
        if date_filter:
            search_query += f" published on {date_filter}"

        if section_filter:
            search_query += f" from the {section_filter} section"

        if sentiment_filter:
            search_query += f" with a sentiment of {sentiment_filter}"

        # Query the index using the query engine
        response = query_engine.query(search_query)

        # Prepare the response
        result = []
        for r in response:
            result.append({
                "Title": r.metadata["Title"],
                "Section": r.metadata["Section"],
                "URL": r.metadata["Url"],
                "Published Date": r.metadata["date"],
                "Summary": r.text[:200]  # Provide a brief summary
            })

        return jsonify({"results": result}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 400

/bin/bash: line 1: ngrok: command not found


In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(app)  # This will use ngrok to tunnel the app

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from datetime import datetime

# Initialize Flask app
app = Flask(__name__)

# Integrate Flask with ngrok
run_with_ngrok(app)

# Real-time indexing: Add articles to the database
@app.route("/articles", methods=["POST"])
def add_article():
    try:
        # Get JSON data from the request
        data = request.get_json()

        # Extract relevant fields
        title = data.get("title")
        section = data.get("section")
        url = data.get("url")
        published_date = data.get("published_date")
        content = data.get("content")

        # Store article (you can implement the actual storage here)
        # Just an example, no actual storage logic

        return jsonify({"message": "Article added successfully!"}), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 400


# Searching articles: Query articles with optional filters
@app.route("/search", methods=["GET"])
def search_articles():
    try:
        query = request.args.get("query", "")
        date_filter = request.args.get("date", None)
        section_filter = request.args.get("section", None)
        sentiment_filter = request.args.get("sentiment", None)

        # Create search query (you can modify the logic here)
        search_query = f"Search articles for: '{query}'"

        return jsonify({"search_query": search_query}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 400


# Run the Flask app with ngrok
if __name__ == "__main__":
    app.run(debug=True)


TypeError: run_with_ngrok.<locals>.new_run() got an unexpected keyword argument 'debug'

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from datetime import datetime

# Initialize Flask app
app = Flask(__name__)

# Integrate Flask with ngrok
run_with_ngrok(app)

# Set the debug mode separately
app.debug = True

# Real-time indexing: Add articles to the database
@app.route("/articles", methods=["POST"])
def add_article():
    try:
        # Get JSON data from the request
        data = request.get_json()

        # Extract relevant fields
        title = data.get("title")
        section = data.get("section")
        url = data.get("url")
        published_date = data.get("published_date")
        content = data.get("content")

        # Store article (you can implement the actual storage here)
        # Just an example, no actual storage logic

        return jsonify({"message": "Article added successfully!"}), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 400


# Searching articles: Query articles with optional filters
@app.route("/search", methods=["GET"])
def search_articles():
    try:
        query = request.args.get("query", "")
        date_filter = request.args.get("date", None)
        section_filter = request.args.get("section", None)
        sentiment_filter = request.args.get("sentiment", None)

        # Create search query (you can modify the logic here)
        search_query = f"Search articles for: '{query}'"

        return jsonify({"search_query": search_query}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 400


# Run the Flask app with ngrok
if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
Exception in thread Thread-17:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    resp

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from datetime import datetime

# Initialize Flask app
app = Flask(__name__)

# Integrate Flask with ngrok
run_with_ngrok(app)

# Real-time indexing: Add articles to the database
@app.route("/articles", methods=["POST"])
def add_article():
    try:
        # Get JSON data from the request
        data = request.get_json()

        # Extract relevant fields
        title = data.get("title")
        section = data.get("section")
        url = data.get("url")
        published_date = data.get("published_date")
        content = data.get("content")

        # Store article (you can implement the actual storage here)
        # Just an example, no actual storage logic

        return jsonify({"message": "Article added successfully!"}), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 400


# Searching articles: Query articles with optional filters
@app.route("/search", methods=["GET"])
def search_articles():
    try:
        query = request.args.get("query", "")
        date_filter = request.args.get("date", None)
        section_filter = request.args.get("section", None)
        sentiment_filter = request.args.get("sentiment", None)

        # Create search query (you can modify the logic here)
        search_query = f"Search articles for: '{query}'"

        return jsonify({"search_query": search_query}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 400


# Run the Flask app with ngrok (no debug=True argument)
if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-18:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           